# 6. 텍스트와 시퀀스를 위한 딥러닝

### 시퀀스 데이터를 처리하는 기본적인 딥러닝 모델
- 순환 신경망
- 1D 컨브넷

## 6.1 텍스트 데이터 다루기

In [2]:
# 텍스트의 벡터화
# 1. 텍스트를 단어로 나누고 각 단어를 하나의 벡터로 변환
# 2. 텍스트를 문자로 나누고 각 문자를 하나의 벡터로 변환
# 3. 텍스트에서 단어나 문자의 n-gram을 추출하여 각 n-gram을 하나의 벡터로 변환
#   여기서 n-gra은 연속된 단어나 문자의 그룹
# 토큰 : 단어 or 문자 or n-gram

import numpy as np

samples = ['The cat sat on the mat.','The dog ate my homework.']

token_index={}
for sample in samples:
    for word in sample.split():
        if word not in token_index:
            token_index[word] = len(token_index)+1

max_length=10

results = np.zeros(shape=(len(samples),
                          max_length,
                          max(token_index.values())+1))

for i, sample in enumerate(samples):
    for j,word in list(enumerate(sample.split()))[:max_length]:
        index = token_index.get(word)
        results[i,j,index]=1.

array([[[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0